In [1]:
from utils import *
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


1. Считываем данные

In [2]:
VOXEL_GRID_SIZE = 512

In [3]:
files = [f for f in os.listdir("data") if f.endswith('.x')]

In [4]:
meshes = []
for f in files:
    print(f)
    filepath = os.path.abspath(f'data\\{f}')
    raw_data = parse_x_file(filepath)

    # Переведём координаты точек из локальной системы координат в систему координат камеры
    # world_transform = apply_transformation(raw_data.get('vertices'), raw_data.get('frame_matrix')) # Даёт худшие результаты
    world_transform = raw_data.get('vertices')
    transformed = apply_transformation(world_transform, raw_data.get('rv_matrix'))
    meshes.append(transformed)

teapot_1.x
num_vertices: 200310
num_faces: 379304
num_uvs: 200310
teapot_2.x
num_vertices: 190307
num_faces: 358169
num_uvs: 190307


2. Объединяем меши в одну вокселную структуру

In [5]:
# Чтобы получить больше информации о процессе объединения мешей
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

In [6]:
# Объединяем меши в один PointCloud
pcd, voxel_size = merge_meshes(meshes, VOXEL_GRID_SIZE, mcd_coarse_scale=60, mcd_fine_scale=12, down_sample=True)

[Open3D DEBUG] [Visualizer] Creating window.
[Open3D DEBUG] GLFW init.
[Open3D DEBUG] Add geometry and update bounding box to [(-0.9998, -83.8568, -51.5618) - (123.1499, 119.4377, 49.6524)]
[Open3D DEBUG] Add geometry and update bounding box to [(-0.9998, -83.8568, -83.0561) - (123.8560, 119.4377, 60.6963)]
[Open3D DEBUG] Global colormap init.
[Open3D DEBUG] [Visualizer] Destroying window.
[Open3D DEBUG] GLFW destruct.
[Open3D DEBUG] ICP Iteration #0: Fitness 0.9198, RMSE 7.7567
[Open3D DEBUG] ICP Iteration #1: Fitness 0.9249, RMSE 7.1981
[Open3D DEBUG] ICP Iteration #2: Fitness 0.9299, RMSE 6.9502
[Open3D DEBUG] ICP Iteration #3: Fitness 0.9323, RMSE 6.7491
[Open3D DEBUG] ICP Iteration #4: Fitness 0.9337, RMSE 6.6078
[Open3D DEBUG] ICP Iteration #5: Fitness 0.9348, RMSE 6.5126
[Open3D DEBUG] ICP Iteration #6: Fitness 0.9359, RMSE 6.4619
[Open3D DEBUG] ICP Iteration #7: Fitness 0.9367, RMSE 6.4281
[Open3D DEBUG] ICP Iteration #8: Fitness 0.9374, RMSE 6.4043
[Open3D DEBUG] ICP Iteration

In [7]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([pcd])

[Open3D DEBUG] [Visualizer] Creating window.
[Open3D DEBUG] GLFW init.
[Open3D DEBUG] Add geometry and update bounding box to [(-1.1612, -83.8371, -51.5121) - (125.4540, 119.8893, 51.6128)]
[Open3D DEBUG] [Visualizer] Destroying window.
[Open3D DEBUG] GLFW destruct.


3. Метод Марширующих кубиков

In [8]:
voxel_grid, origin, scale = pcd_to_voxel_grid(np.asarray(pcd.points), grid_size=VOXEL_GRID_SIZE, apply_filter=False)

In [9]:
# Построим меш на воксельной структуре
vertices, faces, normals, values = marching_cubes(voxel_grid, voxel_size, level=0.0001)
# Переведём координаты в исходный размер
vertices = vertices / (VOXEL_GRID_SIZE - 1) * scale + origin

In [10]:
# Создадим меш на основе вершин и треугольников
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(faces)

In [11]:
# Уберём лишнее если есть
mesh.remove_duplicated_vertices()
mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()
mesh.remove_non_manifold_edges()
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()

[Open3D DEBUG] [RemoveDuplicatedVertices] 0 vertices have been removed.
[Open3D DEBUG] [RemoveDuplicatedVertices] 0 vertices have been removed.
[Open3D DEBUG] [RemoveUnreferencedVertices] 0 vertices have been removed.
[Open3D DEBUG] [RemoveDegenerateTriangles] 0 triangles have been removed.
[Open3D DEBUG] [RemoveDuplicatedTriangles] 0 triangles have been removed.


TriangleMesh with 834736 points and 1744530 triangles.

In [12]:
# Пересчитаем нормали
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()
mesh.normalize_normals()

TriangleMesh with 834736 points and 1744530 triangles.

In [13]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([mesh])

[Open3D DEBUG] [Visualizer] Creating window.
[Open3D DEBUG] GLFW init.
[Open3D DEBUG] Add geometry and update bounding box to [(-1.1612, -83.8371, -51.5121) - (49.1126, -2.9456, -10.5654)]
[Open3D DEBUG] [Visualizer] Destroying window.
[Open3D DEBUG] GLFW destruct.


3.2 Метод Пуассона

In [106]:
# Построим воксельную структуру из откалиброванных точек всех мешей
voxels, origin, _ = build_voxel_grid(meshes, grid_size=VOXEL_GRID_SIZE)

In [17]:
# Запустим метод Пуассона
poisson, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson( pcd=pcd, depth=9)

[Open3D DEBUG] Input Points / Samples: 390617 / 160066
[Open3D DEBUG] #   Got kernel density: 0.07599997520446777 (s), 365.3125 (MB) / 480.3828125 (MB) / 484 (MB)
[Open3D DEBUG] #     Got normal field: 0.6400001049041748 (s), 404.890625 (MB) / 480.3828125 (MB) / 484 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.137608e-06 / 8.349860e-01
[Open3D DEBUG] #       Finalized tree: 0.5130000114440918 (s), 449.40234375 (MB) / 480.3828125 (MB) / 484 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.37600016593933105 (s), 427.79296875 (MB) / 480.3828125 (MB) / 484 (MB)
[Open3D DEBUG] #Set point constraints: 0.11299991607666016 (s), 421.58203125 (MB) / 480.3828125 (MB) / 484 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 1272650 / 1453336 / 1121
[Open3D DEBUG] Memory Usage: 421.582 MB
[Open3D DEBUG] # Linear system solved: 1.128999948501587 (s), 464.65625 (MB) / 480.3828125 (MB) / 484 (MB)
[Open3D DEBUG] Got average: 0.01900005340576172 (s), 413.90234375 (MB) / 480.3828125 (MB) /

In [20]:
# Убрём шум
vertices_to_remove = densities < np.quantile(densities, 0.02)
poisson.remove_vertices_by_mask(vertices_to_remove)

# Пересчитаем нормали
poisson.compute_vertex_normals()
poisson.compute_triangle_normals()
poisson.normalize_normals()

RuntimeError: [Open3D Error] (void __cdecl open3d::geometry::TriangleMesh::RemoveVerticesByMask(const class std::vector<bool,class std::allocator<bool> > &)) D:\a\Open3D\Open3D\cpp\open3d\geometry\TriangleMesh.cpp:1572: vertex_mask has a different size than vertices_


In [21]:
# Посмотрим на результат
o3d.visualization.draw_geometries([poisson])

[Open3D DEBUG] [Visualizer] Creating window.
[Open3D DEBUG] GLFW init.
[Open3D DEBUG] Add geometry and update bounding box to [(-3.3386, -94.0503, -51.4145) - (135.7395, 122.1096, 53.1562)]
[Open3D DEBUG] [Visualizer] Destroying window.
[Open3D DEBUG] GLFW destruct.


3.3 Метод Альфа-формы (Alpha shapes)

In [39]:
alpha = voxel_size * 0.8
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

[Open3D DEBUG] [CreateFromPointCloudAlphaShape] ComputeDelaunayTetrahedralization
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] done ComputeDelaunayTetrahedralization
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] init triangle mesh
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] done init triangle mesh
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] add triangles from tetras that satisfy constraint
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] done add triangles from tetras that satisfy constraint
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] remove triangles within the mesh
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] done remove triangles within the mesh
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] remove duplicate triangles and unreferenced vertices
[Open3D DEBUG] [RemoveDuplicatedTriangles] 0 triangles have been removed.
[Open3D DEBUG] [RemoveUnreferencedVertices] 46984 vertices have been removed.
[Open3D DEBUG] [CreateFromPointCloudAlphaShape] done remove duplicate t

4. Сохранение полученного меша

In [18]:
mesh.triangles

std::vector<Eigen::Vector3i> with 1744530 elements.
Use numpy.asarray() to access data.

In [19]:
write_mesh_to_x(mesh, 'output_mesh.x', save_normals=False, save_texture=False)

Vertices: 100%|██████████| 835k/835k [00:06<00:00, 120k points/s] 
Faces: 100%|██████████| 1.74M/1.74M [00:05<00:00, 311k triangles/s]
